In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import streamlit as st
from openai import OpenAI
from elevenlabs import generate, set_api_key, save
from did import D-ID

# Load API Keys from Streamlit secrets
openai_api_key = st.secrets["OPENAI_API_KEY"]
elevenlabs_api_key = st.secrets["ELEVENLABS_API_KEY"]
did_api_key = st.secrets["DID_API_KEY"]

# Initialize API clients
client = OpenAI(api_key=openai_api_key)
set_api_key(elevenlabs_api_key)
did_client = D-ID(api_key=did_api_key)

# Optional: Summarize your Kaggle notebook content
NOTEBOOK_SUMMARY = """
This notebook initializes numpy and pandas, lists all files under /kaggle/input, and sets up data exploration and ML preprocessing. It operates within Kaggle's working and temp directories with a 20GB limit.
"""

# Function to generate AI, voice, and video response
def agi_voice_agent(user_input):
    prompt = f"Notebook Summary: {NOTEBOOK_SUMMARY}\nUser Question: {user_input}\nAnswer:"
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert AI agent that explains Kaggle notebooks."},
            {"role": "user", "content": prompt}
        ]
    )
    text_output = response.choices[0].message.content

    # Generate voice using ElevenLabs
    audio = generate(text=text_output, voice="Bella", model="eleven_multilingual_v2")
    save(audio, "output.mp3")

    # Generate talking avatar video via D-ID
    video_url = did_client.talk(
        script=text_output,
        driver_url="https://create.d-id.com/avatar/default"  # Optional: customize avatar URL
    )

    return text_output, "output.mp3", video_url['result_url']


# --- Streamlit UI ---
st.set_page_config(page_title="🤖 AGI Voice Agent for Kaggle Notebooks")
st.title("🤖 AGI Voice Agent for Kaggle Notebooks")
st.write("Ask any question about your Kaggle notebook and get text, voice, and video response!")

user_input = st.text_input("What do you want to ask the AGI Notebook?")

if user_input:
    with st.spinner("Generating response..."):
        text, audio_file, video_link = agi_voice_agent(user_input)

        st.subheader("Text Answer")
        st.write(text)

        st.subheader("Voice Answer")
        st.audio(audio_file)

        st.subheader("Talking Video Answer")
        st.video(video_link)

SyntaxError: invalid syntax (598704412.py, line 4)

In [ ]:
OPENAI_API_KEY = "sk-xxxxxx"
ELEVENLABS_API_KEY = "xxxxxx"
DID_API_KEY = "xxxxxx"

In [ ]:
import streamlit as st
from openai import OpenAI
from elevenlabs import generate, set_api_key, save
import requests
import json
import base64

# Load API Keys from Streamlit secrets
openai_api_key = st.secrets["OPENAI_API_KEY"]
elevenlabs_api_key = st.secrets["ELEVENLABS_API_KEY"]
did_api_key = st.secrets["DID_API_KEY"]

# Initialize API clients
client = OpenAI(api_key=openai_api_key)
set_api_key(elevenlabs_api_key)

NOTEBOOK_SUMMARY = """
This notebook initializes numpy and pandas, lists all files under /kaggle/input, and sets up data exploration and ML preprocessing.
"""

# Function to call D-ID API to generate video
def generate_did_video(text_output):
    url = "https://api.d-id.com/talks"
    
    headers = {
        "Authorization": f"Basic {did_api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "script": {
            "type": "text",
            "subtitles": "false",
            "provider": {
                "type": "microsoft",
                "voice_id": "en-US-JennyNeural"
            },
            "ssml": "false",
            "input": text_output
        },
        "source_url": "https://create.d-id.com/models/amy-high-voltage"  # Customize avatar model
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        talk_id = response.json()['id']
        # Construct the video URL
        video_url = f"https://api.d-id.com/talks/{talk_id}/video"
        return video_url
    else:
        return "Failed to generate video."

# AGI Voice Agent function
def agi_voice_agent(user_input):
    prompt = f"Notebook Summary: {NOTEBOOK_SUMMARY}\nUser Question: {user_input}\nAnswer:"
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI agent that explains Kaggle notebooks."},
            {"role": "user", "content": prompt}
        ]
    )
    text_output = response.choices[0].message.content

    # Generate voice using ElevenLabs
    audio = generate(text=text_output, voice="Bella", model="eleven_multilingual_v2")
    save(audio, "output.mp3")

    # Generate D-ID talking avatar video
    video_url = generate_did_video(text_output)

    return text_output, "output.mp3", video_url


# --- Streamlit UI ---
st.set_page_config(page_title="🤖 AGI Voice Agent for Kaggle Notebooks")
st.title("🤖 AGI Voice Agent for Kaggle Notebooks")
st.write("Ask any question about your Kaggle notebook and get text, voice, and video response!")

user_input = st.text_input("What do you want to ask the AGI Notebook?")

if user_input:
    with st.spinner("Generating response..."):
        text, audio_file, video_link = agi_voice_agent(user_input)

        st.subheader("Text Answer")
        st.write(text)

        st.subheader("Voice Answer")
        st.audio(audio_file)

        st.subheader("Talking Video Answer")
        st.video(video_link)